In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy import stats
from scipy.stats import skew
from math import sqrt
from numpy import mean, var
import copy
from sklearn import preprocessing
import json

In [2]:
df = pd.read_csv("credits.csv")
df.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


- DESANIDACION DE LA COLUMNA CAST

In [3]:
import ast
# convertimos la cadena JSON en una lista de diccionarios

df["cast"] = df["cast"].apply(ast.literal_eval)

# Funcion personalizada para extraer los nombres de los generos

def extract_cast_names(cast_list):
	cast_names = [cast["name"] for cast in cast_list]
	return ", ".join(cast_names)

# Aplicamos la funciona personalizada a cada elemento de la columna genres

df["cast"] = df["cast"].apply(extract_cast_names)

# imprimimos el dataframe con la columna de generos normalizados
df.head(3)

,cast,crew,id
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


In [4]:
# creamos una nueva columna para los registros duplicados
df["actor_individual"] = df["cast"].str.split(", ") 
# duplicamos los registros con el metodo explode()
df = df.explode("actor_individual").reset_index(drop=True)
# Eliminamos la primer columna que creamos actor_individual
df.drop("cast", axis=1, inplace=True)
# imprimimos el dataframe con la columna de actores normalizados
df.head(3)

,crew,id,actor_individual
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,Tom Hanks
1,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,Tim Allen
2,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,Don Rickles


In [5]:
df.shape

(565219, 3)

In [6]:
actor_csv = pd.DataFrame(data= df, columns=['id', 'actor_individual'])


In [7]:
actor_csv.head(3)

,id,actor_individual
0,862,Tom Hanks
1,862,Tim Allen
2,862,Don Rickles


In [8]:
actor_csv.to_csv("actores_limpio.csv",index=False)

En este punto ya se tiene un dataset con la informacion de los actores y el id de la pelicula

- NORMALIZACION DE LA COLUMNA CREW para extraer los directores

In [9]:
# volvemos a abrir el archivo credits
df2 = pd.read_csv("credits.csv")
df2.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


In [10]:
import ast
import re

# definimos una expresión regular para extraer el nombre del director de la columna "crew"

# 'Director', 'name'
# 'Director', 'name'
pattern = r"'Director', 'name': '([^']*)'"

# utilizamos el nombre de la coleccion utilizando el pattern
df2["crew"] = df2["crew"].str.extract(pattern, flags=re.IGNORECASE)

In [11]:
df2.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",John Lasseter,862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...",Joe Johnston,8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...",Howard Deutch,15602


In [12]:
# se verifican los valores nuelos de la comuna crew
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    44416 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [13]:
# se encontraron ese numero de valores nulos
df2["crew"].isnull().sum()

1060

In [14]:
# Se eliminan los valores nulos 
df2 = df2.dropna(subset = ["crew"])
# se comprueba nuevamente que los valores nulos ya no estan en el dataframe
df["crew"].isnull().sum()

0

In [16]:
# se crea un nuevo dataframe solo con las columnas id y Crew
director_csv = pd.DataFrame(data= df2, columns=['id', 'crew'])

In [18]:
director_csv.to_csv("director_limpio.csv",index=False)